In [ ]:
from tensorflow.keras.models import load_model
import mnist
import dnn
import os.path
import graph_tool as gt
import nn2graph
import numpy as np
import pandas as pd
import statsmodels.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

#### Build or load model

In [ ]:
load_saved_model = True
if load_saved_model and os.path.exists('data/outputs/mnist_dnn_4x64_10.h5'):
    model = load_model('data/outputs/mnist_dnn_4x64_10.h5')
else:
    model = dnn.build_dense(X_train.shape[-1], hidden_dims=64)
    model.fit(mnist.X_train, mnist.y_train, batch_size=128, epochs=10, validation_data=(mnist.X_test, mnist.y_test))
    model.save('data/outputs/mnist_dnn_4x64_10.h5')

In [ ]:
print(model.evaluate(mnist.X_test, mnist.y_test, verbose=0))
y_pred = model.predict(mnist.X_test)

#### Convert neural network to graph

In [ ]:
from tensorflow.keras.layers import Dense
N = 1000
rand_inds = mnist.subsample_test(N)
layer_sizes = [mnist.X_test.shape[-1]] + [layer.units for layer in model.layers if isinstance(layer, Dense)]
g_full, g_xs = nn2graph.dense_activations_to_graph(model, mnist.X_test[rand_inds])

In [ ]:
from graph_tool.inference.minimize import minimize_nested_blockmodel_dl
min_state = minimize_nested_blockmodel_dl(g_max)
print(min_state)

In [ ]:
min_state.draw()

In [ ]:
x = min_state.get_bs()[0]
plt.imshow(x[:784].reshape((28,28)))
plt.colorbar()
plt.show()
plt.imshow(X_test[i_max].reshape((28,28)))
plt.show()

In [ ]:
import pandas as pd

bs = min_state.get_bs()[0]
df = pd.DataFrame.from_dict({'group': bs, 'node label': list(range(len(bs))), 'layer': g_max.vp['layer'].a})
sns.catplot(x='group', y='node label', data=df, hue='layer')

In [ ]:
from graph_tool.inference.minimize import minimize_nested_blockmodel_dl
min_state = minimize_nested_blockmodel_dl(g_min)
print(min_state)
min_state.draw(layout='sfdp')

In [ ]:
plt.scatter(range(g_min.num_vertices()), min_state.get_bs()[0])